In [2]:
import os
from langchain_community.document_loaders import JSONLoader

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import json
from pathlib import Path
from pprint import pprint
from srn_data_collector.parse_esrs_requirements.parse_esrs_for_rag import parse_data
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp


In [3]:
file_pth = open('/home/ppradhan/openai.txt')
ls_file = open('/home/ppradhan/langsmith.txt')
os.environ["OPENAI_API_KEY"] = file_pth.read().strip()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ls_file.read().strip()

#cuBLAS settings for llama inference
os.environ["CMAKE_ARGS"] = "-DLLAMA_BLAS=on -DLLAMA_BLAS_VENDOR=OpenBLAS"
os.environ["FORCE_CMAKE"] = "1"

In [4]:
os.getcwd()

'/home/ppradhan/Documents/my_dev/esrs_data_collection/srn_data_collector/data-analysis'

In [5]:
esrs_parsed_json = '../parse_esrs_requirements/esrs_requirement_main.json'
esrs_sample_json = '../parse_esrs_requirements/query_req_data.json'
# recommendation_response_path = '/cluster/home/srn_storage/sample_retriever_recommendations'
sample_pdf_raw_parsed_json_path = 'sample_raw_pdf.json'
sample_pdf_formatted_path = '../parse_esrs_requirements/docu_data.json'

### Loading requirements into vectorDB

In [ ]:
# esrs_parsed = parse_data(esrs_parsed_json)
# with open(esrs_sample_json, 'w') as f:
#     json.dump(esrs_parsed,f)

# esrs_parsed

# esrs_parsed.keys()

# main_categories_map = {"environmental": ['climate change', 'pollution', 'water and marine resources', 'biodiversity and ecosystems'],
#                       "social": ['circular economy', 'own workforce', 'workers in the value chain', 'affected communities', 'consumers and end-users'],
#                       "global" : ["business conduct"]}

# def q_builder_metadata_func(record: dict, metadata: dict) -> dict:

#     metadata["label"] = record.get("label")

#     return metadata

# loader = JSONLoader(
#     file_path=esrs_sample_json,
#     jq_schema=' ."climate change"[], ."own workforce"[], ."business conduct"[]',
#     text_content=False,
#     content_key='text',
#     metadata_func=q_builder_metadata_func)

# query_data = loader.load()

# len(query_data)

# query_data[0].page_content

# query_data[0].metadata

# # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# # splits_query = text_splitter.split_documents(query_data)
# # vectorstore_query = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# vectorstore_query = Chroma.from_documents(documents=query_data, embedding=OpenAIEmbeddings())

# # Retrieve and generate using the relevant snippets of the blog.
# query_retriever = vectorstore_query.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# query_retriever.invoke("Query about Climate change, GHG emissions")[:5]

### Loading entire document in vectorDB

In [6]:
with open(sample_pdf_raw_parsed_json_path, 'r') as f:
    sample_doc_dict = json.load(f)

In [7]:
doc_formatted_data = {"doc_data" : []}
for doc_ref, blob_list in sample_doc_dict.items():
    for blob_id, blob_data in enumerate(blob_list):
        doc_formatted_data['doc_data'].append({"text": blob_data['text'], "blob_id": blob_id, "doc_ref": doc_ref})

In [8]:
with open(sample_pdf_formatted_path, 'w') as f:
    json.dump(doc_formatted_data, f)

In [9]:
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["blob_id"] = record.get("blob_id")
    metadata["doc_ref"] = record.get("doc_ref")

    # if "source" in metadata:
    #     source = metadata["source"].split("/")
    #     source = source[source.index("langchain"):]
    #     metadata["source"] = "/".join(source)

    return metadata

In [12]:
doc_loader = JSONLoader(
    file_path=sample_pdf_formatted_path,
    jq_schema='.doc_data[]',
    text_content=False,
    content_key='text',
    metadata_func=metadata_func)

doc_data = doc_loader.load()

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits_doc = text_splitter.split_documents(doc_data)
vectorstore_doc = Chroma.from_documents(documents=splits_doc, embedding=OpenAIEmbeddings())
vectorstore_doc = Chroma.from_documents(documents=doc_data, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever_doc = vectorstore_doc.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [14]:
doc_data[0]

Document(page_content='202\nANNUAL \nREPORT', metadata={'source': '/home/ppradhan/Documents/my_dev/esrs_data_collection/srn_data_collector/parse_esrs_requirements/docu_data.json', 'seq_num': 1, 'blob_id': 0, 'doc_ref': '1'})

In [ ]:
# Creating a prompt template 
sample_requirement = """
AR 43. When preparing the information on gross Scope 2 GHG emissions required under 
paragraph 46, the undertaking shall:
(a) consider the principles and requirements of the GHG Protocol Scope 2 Guidance 
(version 2015 or the latest one); it may also consider corresponding requirements 
for the quantification of indirect GHG emissions from imported energy in ISO 
14064-1:2018;
(b) include purchased or acquired electricity, steam, heat, and cooling consumed by 
the undertaking; 
(c) avoid double counting of GHG emissions reported under Scope 1 or 3;
(d) apply the location-based and market-based methods to calculate Scope 2 GHG 
emissions;
Note: Location-based method quantifies Scope 2 GHG emissions based on 
average energy generation emission factors for defined locations, including local, 
subnational, or national boundaries (GHG Protocol, �Scope 2 Guidance�, 
Glossary, 2015);
Note: Market-based method quantifies Scope 2 GHG emissions based on GHG 
emissions emitted by the generators from which the reporting entity contractually 
purchases electricity bundled with instruments, or unbundled instruments on their 
own (GHG Protocol, �Scope 2 Guidance�, Glossary, 2015); in this case, the 
undertaking may disclose the share of market-based scope 2 GHG emissions 
linked to purchased electricity bundled with instruments such as Guarantee of 
Origins or Renewable Energy Certificates.
(e) disclose biogenic emissions of carbon from the combustion or biodegradation of 
biomass separately from the Scope 2 GHG emissions but include emissions of 
other types of GHG (in particular N2O). In case the emission factors applied do 
not separate the percentage of biomass or biogenic CO2, the undertaking shall 
disclose this. In case GHG emissions other than CO2 (particularly N2O) are not 
available for, or excluded from, location-based grid average emissions factors or 
with the market-based method information, the undertaking shall disclose this;
(f) exclude any purchased, sold or transferred carbon credits or GHG allowances 
from the calculation of Scope 2 GHG emissions; 
(g) adhere to the rules as set out in chapter 7.1 of the GHG Protocol Scope 2 
Guidance (version 2015 or the latest one) and disclose the required information 
accordingly; 
(h) disclose carbon uptakes and emissions (CO2, CO, CH4) from indirect land use 
and land use change separately from the Scope 2 GHG emissions, but include 
emissions of other types of GHG when applicable.
"""

# template = '''
# As an expert in sustainability reports and the new CSRD directive,
# your task is to identify relevant paragraph ids from a list of given
# paragraphs based on a given regulatory Requirement.
# Rank the passages below based on their relevance to the Requirement
# and Sub-Requirement in decreasing order. All the passages
# should be included and listed using identifiers, in
# descending order of relevance. The output format
# should be [] > [], e.g., [1] > [2] > [0]. Only respond
# with the ranking results, do not say any word or explain.
# Requirement: {sample_requirement}
# Passages (with their respective passage ID): 
# {passage_data}
# Response:
# '''

template = '''
[INST] <<SYS>>
As an expert in sustainability reports and the new CSRD directive,
your task is to identify relevant paragraph ids from a list of given
paragraphs based on a given regulatory Requirement.
Rank the passages below based on their relevance to the Requirement
and in decreasing order. All the passages
should be included and listed using identifiers, in
descending order of relevance. The output format
should be [] > [], e.g., [1] > [2] > [0]. Only respond
with the ranking results, do not say any word or explain.
<</SYS>>
Requirement: {sample_requirement}
Passages: {passage_data}
Response:
[/INST]
'''

In [ ]:
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(f"[{doc_idx}]: {doc.page_content}" for doc_idx, doc in enumerate(docs))


rag_chain = (
    {"passage_data": waliml_retriever | format_docs, "sample_requirement": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
type(rag_chain)

In [ ]:
# for chunk in rag_chain.invoke(sample_requirement):
#     print(chunk, end="", flush=True)

rag_chain.invoke("E1.AR43")

In [ ]:
retriever_doc.invoke("GHG emissions")

In [ ]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List


class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        return [Document(page_content=query)]

retriever = CustomRetriever()

retriever.get_relevant_documents("bar")

In [ ]:
retriever.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
from typing import Dict
import json
from tqdm import tqdm

class WaliMLRetriever(BaseRetriever):
    retriever_config: Dict

    def _get_relevant_documents(
        self, query: str,* , run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        result = []
        recommended_doc_dict = self.custom_retriever()
        for compliance_item in recommended_doc_dict:
            if recommended_doc_dict[compliance_item]["section_name"] == query:
                recommendations = recommended_doc_dict[compliance_item].get('recommendations', [])
                for recommendation in tqdm(recommendations, desc=f"processing recommendations for {compliance_item}"):
                    blob_data, _ = recommendation
                    result.append(Document(page_content=blob_data.pop("text"),
                                            metadata=blob_data))

        return result
    def custom_retriever(self):
        if "debug_doc" in self.retriever_config:
            with open(self.retriever_config['debug_doc']['path'], 'r') as f:
                recommended_doc_dict = json.load(f)
        else:
            pass
            # with open(self.retriever_config["wali_ml_training_pkldocs"], 'rb') as f:
            #     training_docs = pkl.load(f)
            # validation_doc_list = [doc["id_"] for doc in training_docs 
            #                     if doc['split'].name == "VALIDATION"]
        return recommended_doc_dict

In [ ]:
retriever_config = {"debug_doc": {"path": "/cluster/home/srn_recommendation_results/fff9295a-2d30-468c-873c-aa4e6c5d7f14.json"}}
waliml_retriever = WaliMLRetriever(retriever_config=retriever_config)
waliml_retriever._get_relevant_documents("E1.AR43",run_manager=None)

In [ ]:
def format_docs(docs):
    return "\n\n".join(f"[{doc_idx}]: {doc.page_content}" for doc_idx, doc in enumerate(docs))


llama_llm = LlamaCpp(
    model_path="/cluster/home/repo/my_llm_experiments/llama-weights/13B/llama-2-13b-chat.ggufv3.q8_0.bin",
    n_gpu_layers=-1,
    n_batch=512,
    n_ctx=5000,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

# llm("The first man on the moon was ... Let's think step by step")


local_rag_chain = (
    {"passage_data": waliml_retriever | format_docs, "sample_requirement": RunnablePassthrough()}
    | custom_rag_prompt
    | llama_llm
    | StrOutputParser()
)

In [ ]:
x = local_rag_chain.invoke("E1.AR43")

In [ ]:
x

In [ ]:
wali_ml_training_pkldocs = "/cluster/home/training_runs/esrs_classifier_final/Parsing/ordinary-priority-000/documents.p"

In [ ]:
import pickle as pkl

with open(wali_ml_training_pkldocs, 'rb') as f:
    training_docs = pkl.load(f)

In [ ]:
training_docs[0]['id_']

In [ ]:
sample_rec_doc = "/cluster/home/srn_recommendation_results/fff9295a-2d30-468c-873c-aa4e6c5d7f14.json"

In [ ]:
with open(sample_rec_doc, 'r') as f:
    rec_doc_dict = json.load(f)

In [ ]:
rec_doc_dict['gross scope 2 GHG emissions (market-based), change to prior fy in %'].keys()

In [ ]:
rec_doc_dict['gross scope 2 GHG emissions (market-based), change to prior fy in %']['section_name']